In [3]:
import requests
from bs4 import BeautifulSoup
products=[]
for i in range (1, 21):
  html1 = requests.get("https://r-komplekt.ru/catalog/kofemashiny_traditsionnye/?PAGEN_1="+str(i)).content.decode('utf-8')
  soup = BeautifulSoup(html1, 'html.parser')
  products.extend(soup.find_all("div", class_="product-line"))
print(len(products))





600


In [4]:
!pip install dash
!pip install dash-bootstrap-components
!pip install dash_core_components






  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 16.1 MB/s eta 0:00:00
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3568 sha256=6effad66a0d5f4f92959715327e107bda50822c949bcccee53cc65bce13f3f23
  Stored in directory: /root/.cache/pip/wheels/b4/33/6d/d0820be98063da218c3206fbad2381cd2db3fbb1a0f0d254b5
Successfully built stringcase
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.4/202.4 kB 4.4 MB/s eta 0:00:00


In [5]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import dash
from dash import dcc, html
import plotly.express as px


In [6]:
links=[]
for product in products:
  links.append("https://r-komplekt.ru" + product.find("a")["href"])


In [7]:

country=[]
brand=[]
price=[]
setting=[]
connection=[]
coffee=[]
boiler_volume=[]
grinder=[]
frother=[]
title=[]
i=1
list1=['Страна', 'Производитель','Установка', 'Подключение','Тип кофе', 'Объём бойлера, л', 'Встроенная кофемолка', 'Капучинатор' ]
for current_link in links:
  print(i)
  html3 = requests.get(current_link).content.decode('utf-8')
  soup = BeautifulSoup(html3, 'html.parser')
  if soup.find( class_="changeShortDescription") is not None and soup.find( class_="price changePrice") is not None:
    title.append(soup.find( class_="changeShortDescription").get_text(strip=True))
    price.append(soup.find( class_="price changePrice").get_text(strip=True))
    tables=soup.find_all('table')
    correct_table=tables[0]
    headers = []
    items = []
    for header in correct_table.find_all('th'):
        headers.append(header.text.strip())
    for line in correct_table.find_all('tr'):
        cells = line.find_all('td')
        row = []
        for cell in cells:
          row.append(cell.text.strip())
        if row:
          items.append(row)
    df = pd.DataFrame(items)
    df=df.set_index(0)
    df=df.fillna("--")
    #df=df.drop(2, axis=1)
    for item in list1:
      if item not in df.index:
        new_values = {col: '—-' for col in df.columns}
        df.loc[item]=new_values
    country.append(df.loc['Страна', 1])
    brand.append(df.loc['Производитель', 1])
    setting.append(df.loc['Установка', 1])
    connection.append(df.loc['Подключение', 1])
    coffee.append(df.loc['Тип кофе', 1])
    boiler_volume.append(df.loc['Объём бойлера, л', 1])
    grinder.append(df.loc['Встроенная кофемолка', 1])
    frother.append(df.loc['Капучинатор', 1])

    i+=1
  else:
    pass


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
71
72
73
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
87
88
89
90
91
92
93
94
95
96
97
98
99
99
100
101
102
103
104
105
106
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
210
211
212
213
214
215
216
217
218
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270


In [107]:

df=pd.DataFrame({"Название": title , "Цена": price, "Страна производства": country, "Бренд": brand, "Тип установки": setting, "Тип подключения": connection, "Тип кофе": coffee, "Объем бойлера, л": boiler_volume, "Кофемолка": grinder, "Капуччинатор": frother})

In [108]:
for i in range(len(brand)):
  brand[i]=brand[i].split('  ')[0]


In [109]:
df

,Название,Цена,Страна производства,Бренд,Тип установки,Тип подключения,Тип кофе,"Объем бойлера, л",Кофемолка,Капуччинатор
0,Кофемашина рожковая MANIFESTA Professionale 2 ...,349 490 руб.,Италия,MANIFESTA,Настольная,Электрическое,молотый,11.5,Нет,Есть
1,Кофемашина рожковая C.M.A. ASTORIA Pratic Avan...,480 000 руб.,Италия,C.M.A. Astoria,Настольная,Электрическое,молотый,10.5,Нет,Есть
2,Кофемашина рожковая NUOVA SIMONELLI Appia Life...,352 314 руб.,Италия,Nuova Simonelli,Настольная,Электрическое,молотый,11,Нет,Есть
3,Кофемашина рожковая QUALITY ESPRESSO FUTURMAT ...,228 901 руб.,Испания,QUALITY ESPRESSO FUTURMAT,Настольная,Электрическое,молотый,5,Нет,Есть
4,Кофемашина рожковая ESTREMO Djua 2 высокие гру...,239 250 руб.,Китай,ESTREMO,Настольная,Электрическое,молотый,11,Нет,Есть
...,...,...,...,...,...,...,...,...,...,...
573,Кофемашина рожковая LA PAVONI GENIALE3VB 3 низ...,644 080 руб.,Швейцария,LA PAVONI,Настольная,Электрическое,молотый,21,Нет,Есть
574,Кофемашина рожковая LA PAVONI DESIDERIO3VB 3 н...,869 120 руб.,Швейцария,LA PAVONI,Настольная,Электрическое,молотый,21,Нет,Есть
575,Кофемашина рожковая LA PAVONI DIA3SV 3 низкие ...,826 440 руб.,Швейцария,LA PAVONI,Настольная,Электрическое,молотый,22.5,Нет,Есть
576,Кофемашина рожковая CREM EX2 Mini 1 высокая гр...,243 500 руб.,Испания,CREM,Настольная,Электрическое,молотый,6,Нет,Есть


In [110]:
df['Цена'] = (df['Цена'].apply(lambda item: ''.join(filter(str.isdigit, item))))
df['Цена'] = pd.to_numeric(df['Цена'], errors='coerce')

In [111]:
df['Цена'].dtype

dtype('int64')

In [112]:
set(df["Страна производства"])

{'Германия',
 'Испания',
 'Италия',
 'Китай',
 'Португалия',
 'Россия',
 'США',
 'Турция',
 'Франция',
 'Швейцария',
 '—-'}

In [113]:
countries_counts=df['Страна производства'].value_counts().to_frame()
countries_counts=countries_counts.drop(index='—-')

print(countries_counts.index)
countries = ['Italy', 'Switzerland', 'Spain', 'Portugal', 'Russia', 'China',
             'Turkey', 'France', 'USA', 'Germany']
countries_counts.index = countries


Index(['Италия', 'Швейцария', 'Испания', 'Португалия', 'Россия', 'Китай',
       'Турция', 'Франция', 'США', 'Германия'],
      dtype='object', name='Страна производства')


In [114]:
countries_counts

,count
Italy,425
Switzerland,40
Spain,38
Portugal,25
Russia,14
China,13
Turkey,10
France,5
USA,4
Germany,2


In [115]:
countries_counts["Страны"]=['Италия', 'Швейцария', 'Испания', 'Португалия', 'Россия', 'Китай',
       'Турция', 'Франция', 'США', 'Германия']

In [116]:
countries_counts

,count,Страны
Italy,425,Италия
Switzerland,40,Швейцария
Spain,38,Испания
Portugal,25,Португалия
Russia,14,Россия
China,13,Китай
Turkey,10,Турция
France,5,Франция
USA,4,США
Germany,2,Германия


In [117]:
df

,Название,Цена,Страна производства,Бренд,Тип установки,Тип подключения,Тип кофе,"Объем бойлера, л",Кофемолка,Капуччинатор
0,Кофемашина рожковая MANIFESTA Professionale 2 ...,349490,Италия,MANIFESTA,Настольная,Электрическое,молотый,11.5,Нет,Есть
1,Кофемашина рожковая C.M.A. ASTORIA Pratic Avan...,480000,Италия,C.M.A. Astoria,Настольная,Электрическое,молотый,10.5,Нет,Есть
2,Кофемашина рожковая NUOVA SIMONELLI Appia Life...,352314,Италия,Nuova Simonelli,Настольная,Электрическое,молотый,11,Нет,Есть
3,Кофемашина рожковая QUALITY ESPRESSO FUTURMAT ...,228901,Испания,QUALITY ESPRESSO FUTURMAT,Настольная,Электрическое,молотый,5,Нет,Есть
4,Кофемашина рожковая ESTREMO Djua 2 высокие гру...,239250,Китай,ESTREMO,Настольная,Электрическое,молотый,11,Нет,Есть
...,...,...,...,...,...,...,...,...,...,...
573,Кофемашина рожковая LA PAVONI GENIALE3VB 3 низ...,644080,Швейцария,LA PAVONI,Настольная,Электрическое,молотый,21,Нет,Есть
574,Кофемашина рожковая LA PAVONI DESIDERIO3VB 3 н...,869120,Швейцария,LA PAVONI,Настольная,Электрическое,молотый,21,Нет,Есть
575,Кофемашина рожковая LA PAVONI DIA3SV 3 низкие ...,826440,Швейцария,LA PAVONI,Настольная,Электрическое,молотый,22.5,Нет,Есть
576,Кофемашина рожковая CREM EX2 Mini 1 высокая гр...,243500,Испания,CREM,Настольная,Электрическое,молотый,6,Нет,Есть


In [118]:
for i in range(len(brand)):
  brand[i]=brand[i].split('  ')[0]
print(set(brand))

{'CIME', 'QUALITY ESPRESSO FUTURMAT', 'CREM', 'SANREMO', 'AUTOMATIC COFFEE MACHINES SRL', 'ROBUSTEZZA', 'LA PAVONI', 'HEYLO', 'FIAMMA', 'CARIMALI', 'ESTREMO', 'RANCILIO', 'WMF', 'DAZHENG', 'ELEKTRA', 'BEZZERA', 'CASADIO', 'SYNESSO', 'DALLA CORTE', 'LA CIMBALI', 'MANIFESTA', 'Victoria Arduino', 'COFFF', 'EMPERO', 'Nuova Simonelli', 'C.M.A. Astoria', 'KIMNELLI', 'NEW COFFEE MACHINES', 'GUFO', 'CONTI', 'MAVAM', 'GEMILAI'}


In [119]:
brand_count=df['Бренд'].value_counts().to_frame()

In [120]:
brand_count=brand_count.reset_index()
display(brand_count)

,Бренд,count
0,Victoria Arduino,89
1,Nuova Simonelli,51
2,RANCILIO,43
3,DALLA CORTE,41
4,LA PAVONI,40
5,SANREMO,38
6,CREM,35
7,ROBUSTEZZA,33
8,CARIMALI,28
9,FIAMMA,25


In [121]:
brand_count.loc[brand_count['count']<6, "Бренд" ]="Другие бренды"

In [122]:
display(brand_count)

,Бренд,count
0,Victoria Arduino,89
1,Nuova Simonelli,51
2,RANCILIO,43
3,DALLA CORTE,41
4,LA PAVONI,40
5,SANREMO,38
6,CREM,35
7,ROBUSTEZZA,33
8,CARIMALI,28
9,FIAMMA,25


In [123]:
grouped_machines1 = df.groupby('Страна производства')
result = grouped_machines1.agg(
    num_brand=('Бренд', 'nunique'),
    top_brand=('Бренд', lambda x: x.mode()[0]),
    all_brand=('Бренд', lambda x: ', '.join(x.unique())),
    avg_price=('Цена', 'mean')
).reset_index()
display(result)

,Страна производства,num_brand,top_brand,all_brand,avg_price
0,Германия,1,WMF,WMF,1.390930e+06
1,Испания,2,CREM,"QUALITY ESPRESSO FUTURMAT, CREM",4.525442e+05
2,Италия,16,Victoria Arduino,"MANIFESTA, C.M.A. Astoria, Nuova Simonelli, RO...",8.844971e+05
3,Китай,4,KIMNELLI,"ESTREMO, DAZHENG, KIMNELLI, GEMILAI",3.117000e+05
4,Португалия,1,FIAMMA,FIAMMA,3.694188e+05
5,Россия,2,NEW COFFEE MACHINES,"GUFO, NEW COFFEE MACHINES",4.432321e+05
6,США,2,MAVAM,"SYNESSO, MAVAM",2.239196e+06
7,Турция,1,EMPERO,EMPERO,2.752634e+05
8,Франция,1,CONTI,CONTI,6.027972e+05
9,Швейцария,1,LA PAVONI,LA PAVONI,4.879491e+05


In [133]:
fig = px.pie(brand_count,
             values='count',
             hole=0.4,
             names='Бренд',
             hover_data=['count'],
             color_discrete_sequence=px.colors.sequential.Magma,
             labels={
                 'count': 'количество',
                 'Бренд': 'бренд кофемашины'
             },
             title='Распределение кофемашин по брендам',
             width=1000,
             height=600
            )


fig.show()

In [126]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd
from sklearn.datasets import load_iris
import dash_bootstrap_components as dbc
from dash import dcc
import dash_core_components as dcc
app = Dash(__name__)

fig = px.histogram(df, x=df['Цена'], nbins=100, title='Гистограмма цен на кофемашины', color_discrete_sequence=["#636EFA"])
app.layout = html.Div([
    dcc.Graph(
        id='price_histogram',
        figure=fig
    )
])

if __name__ == '__main__':

  app.run(debug=True)

<IPython.core.display.Javascript object>

In [127]:

app = Dash(__name__)

fig2 = px.choropleth(countries_counts,
                    locations=countries_counts.index,
                    locationmode="country names",
                    color=countries_counts["count"],
                    color_continuous_scale="plasma",
                    title="Количество наблюдений по странам",
                    projection='natural earth',

                    width=800,
                    height=500)

app.layout = html.Div([
    html.H1("Количество наблюдений по странам"),
    dcc.Graph(
        id='world_map',
        figure=fig2
    )
])
if __name__ == '__main__':

  app.run(debug=True)

<IPython.core.display.Javascript object>

In [128]:
top_countries = countries_counts.nlargest(5, 'count')["Страны"].tolist()

In [129]:
top_countries

['Италия', 'Швейцария', 'Испания', 'Португалия', 'Россия']

In [132]:

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
dashboard_layout = dbc.Container([
    html.H1("Дашборд с информацией про кофемашины", className='text-center text-primary mb-4'),
    html.H3("Распределение кофемашин по странам", className='text-center mb-2'),
    dbc.Row([
        dbc.Col(dcc.Graph(id='world-map', figure=fig2), width=12)
    ], className='mb-4'),
    html.H3("Гистограмма цен на кофемашины", className='text-center mb-2'),
    dbc.Row([
        dbc.Col(dcc.Graph(id='price_histogram', figure=fig), width=12)
    ], className='mb-4'),
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader('Выберите страну'),
                dbc.CardBody([
                    dcc.Dropdown(
                        id='country-dropdown',
                        options=[{'label': country, 'value': country} for country in top_countries],
                        placeholder='Выберите страну',
                    )
                ])
            ])
        ], width=4),
        dbc.Col([
            dbc.Card([
                dbc.CardHeader('Информация о кофемашинах'),
                dbc.CardBody([
                    html.P(id="top_brand", className="card-text"),
                    html.P(id="avg_price", className="card-text"),
                    html.P(id="num_brand", className="card-text"),
                    html.P(id='all_brand', className="card-text")
                ])
            ])
        ], width=8)
    ])
], fluid=True)

@app.callback(
    [Output("top_brand", "children"),
     Output("avg_price", "children"),
     Output("num_brand", "children"),
     Output("all_brand", "children")],
    [Input("country-dropdown", "value")]
)
def update_info(selected_country):
    if not selected_country:
        return "Выберите страну", "", "", ""

    filtered_data = result[result["Страна производства"] == selected_country]

    if filtered_data.empty:
        print("AAAAAA")
        return "Нет данных", "Нет данных", "Нет данных", "Нет данных"

    row = result[result["Страна производства"] == selected_country].iloc[0]
    return (
        f"Топовый бренд: {row['top_brand']}",
        f"Средняя цена: {row['avg_price']} руб.",
        f"Количество брендов: {row['num_brand']}",
        f"Все бренды: {row['all_brand']}"
    )


app.layout = dashboard_layout

if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>